In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
file_path = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/文件_XsensMVN输出文档/XsensMTw_ExampleData.txt"

In [4]:
df = pd.read_csv(file_path,skiprows=12,sep="\t")

In [6]:
df.columns

Index(['PacketCounter', 'Year', 'Month', 'Day', 'Second', 'UTC_Nano',
       'UTC_Year', 'UTC_Month', 'UTC_Day', 'UTC_Hour', 'UTC_Minute',
       'UTC_Second', 'UTC_Valid', 'Acc_X', 'Acc_Y', 'Acc_Z', 'FreeAcc_E',
       'FreeAcc_N', 'FreeAcc_U', 'AccHR_X', 'AccHR_Y', 'AccHR_Z', 'Gyr_X',
       'Gyr_Y', 'Gyr_Z', 'GyrHR_X', 'GyrHR_Y', 'GyrHR_Z', 'Mag_X', 'Mag_Y',
       'Mag_Z', 'VelInc_X', 'VelInc_Y', 'VelInc_Z', 'OriInc_q0', 'OriInc_q1',
       'OriInc_q2', 'OriInc_q3', 'Quat_q0', 'Quat_q1', 'Quat_q2', 'Quat_q3',
       'Roll', 'Pitch', 'Yaw', 'Mat[1][1]', 'Mat[2][1]', 'Mat[3][1]',
       'Mat[1][2]', 'Mat[2][2]', 'Mat[3][2]', 'Mat[1][3]', 'Mat[2][3]',
       'Mat[3][3]', 'Latitude', 'Longitude', 'Altitude', 'Vel_E', 'Vel_N',
       'Vel_U', 'GyrTemp_X', 'GyrTemp_Y', 'GyrTemp_Z'],
      dtype='object')

## 欧拉角转为旋转矩阵

In [24]:
from numpy import cos as cos
from numpy import sin as sin
def euler2matrix(phi,theta,psi):
    '''
    功能：
    根据欧拉角计算旋转矩阵

    参数：
    phi: 围绕 X 轴旋转的角度
    theta: 围绕 Y 轴旋转角度
    psi: 围绕 Z 轴旋转角度
    单位为 度

    返回: 
    旋转矩阵
    旋转矩阵的顺序为：mat11,mat12,mat13, mat21, mat22, mat23, mat31,mat32,mat33
    '''
    phi_rad = np.deg2rad(phi)
    theta_rad = np.deg2rad(theta)
    psi_rad = np.deg2rad(psi)

    mat11 = cos(theta_rad)*cos(phi_rad)
    mat12 = sin(phi_rad)*sin(theta_rad)*cos(psi_rad) - cos(phi_rad)*sin(psi_rad)
    mat13 = cos(phi_rad)*sin(theta_rad)*cos(psi_rad) + sin(phi_rad)*sin(psi_rad)

    mat21 = cos(theta_rad)*sin(phi_rad)
    mat22 = sin(phi_rad)*sin(theta_rad)*sin(psi_rad) + cos(phi_rad)*cos(psi_rad)
    mat23 = cos(phi_rad)*sin(theta_rad)*sin(psi_rad) - sin(phi_rad)*cos(psi_rad)

    mat31 = -sin(theta_rad)
    mat32 = sin(phi_rad)*cos(theta_rad)
    mat33 = cos(phi_rad)*cos(theta_rad) 

    matrix = np.array([mat11,mat12,mat13,
             mat21, mat22, mat23,
             mat31,mat32,mat33])
    return matrix

In [29]:
matrix_xsens = ['Mat[1][1]','Mat[1][2]','Mat[1][3]',
                'Mat[2][1]','Mat[2][2]','Mat[2][3]',
                'Mat[3][1]','Mat[3][2]','Mat[3][3]']
euler_xsens = ['Roll', 'Pitch', 'Yaw']
free_acc_xsens = ['FreeAcc_E','FreeAcc_N', 'FreeAcc_U']

In [44]:
cal_matrix = pd.DataFrame()
for i, row in df.iterrows():
    matrix = euler2matrix(row["Roll"], row["Pitch"], row["Yaw"])
    cal_matrix.loc[i] = matrix.flatten()

ValueError: cannot set a frame with no defined columns

In [ ]:
data = pd.DataFrame({"roll":[60,90,150],
                     "picth":[90,60,130],
                     "yaw":[70,80,60]})

## 计算自由加速度

In [55]:
matrix_33 = np.array(df[matrix_xsens]).reshape((df.shape[0],3,3))



AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [56]:
df[matrix_xsens]

,Mat[1][1],Mat[1][2],Mat[1][3],Mat[2][1],Mat[2][2],Mat[2][3],Mat[3][1],Mat[3][2],Mat[3][3]
0,-0.031068,-0.141484,0.989453,-0.043483,-0.988802,-0.142757,0.998571,-0.047460,0.024568
1,-0.026836,-0.132383,0.990835,-0.047355,-0.989911,-0.133542,0.998518,-0.050505,0.020296
2,-0.028962,-0.136812,0.990174,-0.045356,-0.989389,-0.138030,0.998551,-0.048908,0.022449
3,-0.028974,-0.136683,0.990191,-0.045188,-0.989415,-0.137898,0.998558,-0.048740,0.022491
4,-0.029006,-0.136731,0.990183,-0.044905,-0.989422,-0.137942,0.998570,-0.048465,0.022559
5,-0.029057,-0.136810,0.990171,-0.044732,-0.989419,-0.138019,0.998576,-0.048303,0.022630
6,-0.029080,-0.137086,0.990132,-0.044473,-0.989393,-0.138289,0.998587,-0.048056,0.022675
7,-0.029062,-0.137271,0.990107,-0.044348,-0.989373,-0.138471,0.998593,-0.047934,0.022666
8,-0.029108,-0.137462,0.990079,-0.044165,-0.989355,-0.138660,0.998600,-0.047763,0.022728
9,-0.029107,-0.137652,0.990053,-0.044060,-0.989333,-0.138847,0.998605,-0.047664,0.022731


## 尝试使用一个数计算自由加速度

In [92]:
# 获取第 800 行的旋转矩阵
matrix_800 = df[matrix_xsens].loc[800]
matrix_33_800 = np.array(matrix_800).reshape(3,3)

In [95]:
# 计算自由加速度
np.dot(matrix_33_800,df[['Acc_X', 'Acc_Y', 'Acc_Z']].loc[800])

array([ 1.02103052, -0.20560013,  9.25991654])

In [96]:
print("第800行的自由加速度为\n",df[free_acc_xsens].loc[800])

第800行的自由加速度为
 FreeAcc_E    1.055726
FreeAcc_N   -0.075794
FreeAcc_U   -0.555010
Name: 800, dtype: float64
